## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [1]:
import mysql.connector

In [3]:
cnx = mysql.connector.connect(
    host = "127.0.0.1",
    user = "root",
    password = "livros"
)

print(cnx)

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

CURSOR BUFFER

A razão é que sem um cursor em buffer, os resultados são carregados "preguiçosamente", o que significa que "fetchone" na verdade busca apenas uma linha do conjunto completo de resultados da consulta. Quando você usar o mesmo cursor novamente, ele reclamará que você ainda tem n-1 resultados (onde n é a quantidade do conjunto de resultados) esperando para serem buscados. No entanto, quando você usa um cursor em buffer, o conector busca TODAS as linhas nos bastidores e você apenas pega uma do conector para que o banco de dados mysql não reclame.

In [4]:
cursor = cnx.cursor(buffered=True)

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [ ]:
# cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [5]:
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)

('clube_do_livro',)
('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [ ]:
import pandas as pd

df_livros = pd.read_csv(r"C:\Users\Microcity\Desktop\PYTHON_MONGO_MYSQL\notebooks\data\dados_transformados.csv")
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,657092eb7e6a7df963ebf0d6,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,657092eb7e6a7df963ebf0d7,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,657092eb7e6a7df963ebf0e9,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,657092eb7e6a7df963ebf0f7,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,657092eb7e6a7df963ebf0f9,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [ ]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [ ]:
df_livros.shape

(742, 13)

In [ ]:
cursor.execute("""
               
CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
                
                PRIMARY KEY (id)
                
                );
               """)

**Selecionando a base de dados para verificar a tabela criada**

In [6]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
    print(tb)



('tb_livros',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [ ]:
# Neste for, utilizaremos 2 variáveis, pois o iterrows retorna dois tipos de conteudo.
# Retorno i -> cada um dos índices do nosso DataFrame.
# Retorno row -> cada um dos dados de cada linha que temos no DataFrame. 
for i, row in df_livros.iterrows():
    print(tuple(row))

('657092eb7e6a7df963ebf0d6', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('657092eb7e6a7df963ebf0d7', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('657092eb7e6a7df963ebf0e9', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('657092eb7e6a7df963ebf0f7', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('657092eb7e6a7df963ebf0f9', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('657092eb7e6a7df963ebf0fc', 'Iniciando em programação', 'livros', 61.73, 0.0, '2021-02-01', 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('657092eb7e6a7df963ebf0

In [ ]:
# transformando com o list comprehension 
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados


[('657092eb7e6a7df963ebf0d6',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('657092eb7e6a7df963ebf0d7',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('657092eb7e6a7df963ebf0e9',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('657092eb7e6a7df963ebf0f7',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('657092eb7e6a7df963ebf0f9',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('657092eb7e6a7df963ebf0fc',
  'Iniciando em progr

In [ ]:
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados)
cnx.commit()

In [ ]:
print(cursor.rowcount, "dados inseridos")

1 dados inseridos


## Visualizando os dados inseridos

In [7]:
cursor.execute("SELECT * FROM dbprodutos.tb_livros;")

for row in cursor:
    print(row)

('657092eb7e6a7df963ebf0d6', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('657092eb7e6a7df963ebf0d7', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('657092eb7e6a7df963ebf0e9', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('657092eb7e6a7df963ebf0f7', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('657092eb7e6a7df963ebf0f9', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('657092eb7e6a7df963ebf0fc', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

In [ ]:
cursor.close()
cnx.close()